In [1]:
data_var = '2024-01-17'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10987,2024-01-17,Europa Wbbl Feminina,14:00,Jurmala F,Siauliai F,18.00,1.01,143.5,1.85,1.85,23.5,2.00,1.72,b5kTH0Sn,0.055556,0.990099,0.540541,0.540541,0.045655,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,46,59,9.70,9.55,0.000,0.000,1.263939,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10988,2024-01-17,Japão B.League,07:05,Diamond Dolphins,Ryukyu,2.07,1.70,157.5,1.87,1.83,1.5,1.91,1.78,UoZlNjql,0.483092,0.588235,0.534759,0.546448,0.071327,207.462,88.518641,0.426674,0.6,1.341641,2.236068,187.23,2.840,1.014840,0.357338,-33.0,103.840,19.980534,0.192417,1.2,1.643168,1.369306,76.38,1.388,0.162388,0.116995,2.0,79,57,2.37,1.34,144.398,104.834,0.138795,0.015289,0.049823,-0.32,-0.064,-16.718750,0.000000,0.0,0.000000,-0.72,-0.144,-4.861111,0.697138,0.6,-0.097138
10989,2024-01-17,Filipinas Copa Commissioners,05:00,Phoenix Fuelmasters,Meralco Bolts,2.57,1.48,192.5,1.87,1.79,3.5,2.06,1.68,21vLKeFN,0.389105,0.675676,0.534759,0.558659,0.064781,219.028,42.217958,0.192751,1.8,1.643168,0.912871,186.24,2.330,0.608400,0.261116,-17.0,168.824,59.215623,0.350754,1.8,1.643168,0.912871,126.44,1.810,0.757793,0.418670,-6.0,96,109,1.94,1.16,187.498,188.084,0.380616,0.030912,0.143690,4.74,0.948,1.656118,0.000000,0.0,0.000000,2.99,0.598,0.802676,0.000000,0.0,0.000000
10990,2024-01-17,Filipinas Copa Commissioners,09:00,Magnolia Hotshots,TNT Tropang Giga,1.33,3.21,195.5,1.86,1.86,-9.5,2.07,1.67,OAuHLy0H,0.751880,0.311526,0.537634,0.537634,0.063406,115.980,15.229077,0.131308,1.8,1.643168,0.912871,110.40,1.224,0.140464,0.114758,17.0,197.926,39.643606,0.200295,1.2,1.643168,1.369306,209.96,2.020,0.619032,0.306451,9.0,80,116,1.38,1.81,0.000,184.230,0.585621,0.000000,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,-2.49,-0.498,-4.437751,0.000000,0.0,0.000000
10991,2024-01-17,Islândia Premier League Feminina,16:15,Keflavik F,Grindavik F,1.36,2.97,146.5,1.85,1.85,-12.5,1.85,1.85,YFmTwnRO,0.735294,0.336700,0.540541,0.540541,0.071994,92.054,13.810224,0.150023,2.4,1.341641,0.559017,68.64,1.156,0.137586,0.119019,93.0,117.464,24.264337,0.206568,2.4,1.341641,0.559017,112.66,1.440,0.421782,0.292904,56.0,66,86,1.04,1.31,0.000,0.000,0.525839,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,2024-01-17,Eua Ncaa,21:00,Loyola (Ill).,UMass,1.73,2.10,145.5,1.80,1.91,-2.5,1.83,1.83,ADnz3RmH,0.578035,0.476190,0.555556,0.523560,0.054225,139.848,61.111270,0.436984,2.4,1.341641,0.559017,233.22,1.922,0.695536,0.361881,51.0,139.900,65.758071,0.470036,1.8,1.643168,0.912871,130.13,1.968,1.257684,0.639067,32.0,78,77,2.99,1.69,107.374,0.000,0.136621,0.041931,0.000000,0.29,0.058,12.586207,0.724174,0.7,-0.024174,0.00,0.000,inf,0.000000,0.0,0.000000
11140,2024-01-17,Eua Ncaa,21:00,LSU,Ole Miss,1.73,2.10,149.5,1.80,1.91,-1.5,1.80,1.91,fN7pXuxQ,0.578035,0.476190,0.555556,0.523560,0.054225,289.560,284.837729,0.983692,2.4,1.341641,0.559017,

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11000,07:05,Japão B.League,Sun Rockers Shibuya,Brave Thunders,1.61,Back Home
11020,16:00,Europa Euroliga Feminina,Lyon F,Schio F,1.54,Back Home
11040,13:30,Europa Liga Letônia,TAL TECH,Rigas Zelli,2.76,Back Home
11062,13:45,Europa Copa Alpe Adria,Kolin,BC Timisoara,2.18,Back Home
11064,21:00,Eua Ncaa,Wofford,East Tennessee St,1.55,Back Home
11090,21:30,Eua Nba,Atlanta Hawks,Orlando Magic,1.69,Back Home
11094,21:30,Eua Nba,New York Knicks,Houston Rockets,1.58,Back Home
11105,14:00,Eslováquia Extraliga,Prievidza,BC Komarno,1.99,Back Home
11122,22:00,Eua Ncaa,Texas State,La Lafayette,1.83,Back Home
11142,21:00,Eua Ncaa,Boston University,Bucknell,1.55,Back Home
